Check we have connection correct. To test this we call get a FHIR Conformance statement from the server

In [13]:
import requests
from requests.auth import HTTPBasicAuth
from dotenv import load_dotenv
load_dotenv()
import os

fhir_password = os.getenv("FHIR_PASSWORD")
fhir_username = os.getenv("FHIR_USERNAME")
server = os.getenv("FHIR_SERVER")

api_url = server + "metadata"
response = requests.get(api_url)
#response.json()

Find a patient. Simple search for a patient named wrexham

In [14]:


api_url = server + "Patient?name=wrexham"
print(api_url)
response = requests.get(api_url, auth=HTTPBasicAuth(fhir_username, fhir_password))
response.json()

https://gen-hie-prod.nwgenomics.nhs.uk/dataplatform/cdr/fhir/r4/Patient?name=wrexham


{'resourceType': 'Bundle',
 'id': '4f2c7cc2-063d-4a04-9918-4c30134dcd54',
 'type': 'searchset',
 'timestamp': '2025-12-24T07:31:19Z',
 'total': 0,
 'link': [{'relation': 'self',
   'url': 'https://gen-hie-prod.nwgenomics.nhs.uk/dataplatform/cdr/fhir/r4/Patient?name=wrexham'}]}

Now lets find diagnostic reports for this patient.

In [15]:
api_url = server + "DiagnosticReport?patient=18"
response = requests.get(api_url, auth=HTTPBasicAuth(fhir_username, fhir_password))
response.json()

{'resourceType': 'Bundle',
 'id': '32d4863d-1c17-482b-89ca-b6133488816a',
 'type': 'searchset',
 'timestamp': '2025-12-24T07:31:20Z',
 'total': 0,
 'link': [{'relation': 'self',
   'url': 'https://gen-hie-prod.nwgenomics.nhs.uk/dataplatform/cdr/fhir/r4/DiagnosticReport?patient=18'}]}

Can also return details about the organisation that created this.

In [16]:
api_url = server + "Organization/5"
response = requests.get(api_url, auth=HTTPBasicAuth(fhir_username, fhir_password))
response.json()

{'resourceType': 'Organization',
 'active': True,
 'id': '5',
 'identifier': [{'system': 'https://fhir.nhs.uk/Id/ods-organization-code',
   'value': 'RX1'}],
 'name': 'NOTTINGHAM UNIVERSITY HOSPITALS NHS TRUST',
 'partOf': {'identifier': {'system': 'https://fhir.nhs.uk/Id/ods-organization-code',
   'value': 'QT1'},
  'reference': 'Organization/6'},
 'type': [{'coding': [{'code': 'RO197',
     'display': 'NHS TRUST',
     'system': 'https://digital.nhs.uk/services/organisation-data-service/CodeSystem/ODSOrganisationRole'}]}],
 'meta': {'lastUpdated': '2025-12-19T17:19:21Z', 'versionId': '2'}}

Example using python to interpret the json and also using [fhirclient](https://github.com/smart-on-fhir/client-py)

In [17]:
import fhirclient.models.diagnosticreport as dr
import pandas as pd

api_url = server + "DiagnosticReport?_count=50"
response = requests.get(api_url, auth=HTTPBasicAuth(fhir_username, fhir_password))
responseJSON = response.json()

print(responseJSON['total'])

reports = []
reportJ = []
for entry in responseJSON['entry']:
    #print(entry['resource'])
    print(entry['resource']['resourceType'], entry['resource']['issued'] )
    report = dr.DiagnosticReport(entry['resource'])
    reportJ.append(entry['resource'])
    reports.append(report)
    for coding in report.code.coding:
        print(coding.code)

145
DiagnosticReport 2025-12-17T11:56:42+00:00
R31.3_WGS
DiagnosticReport 2025-12-17T11:57:06+00:00
TARGET_ARRAY_SNP
DiagnosticReport 2025-12-17T12:25:36+00:00
M80_AML_GX
DiagnosticReport 2025-12-17T12:22:52+00:00
SANGER Family follow-up Rare
DiagnosticReport 2025-12-17T11:57:07+00:00
SANGER Diagnostic Rare
DiagnosticReport 2025-12-17T12:38:42+00:00
TARGET_ARRAY_SNP
DiagnosticReport 2025-12-17T13:04:03+00:00
CCP19v1_R125v4
R125.1
DiagnosticReport 2025-12-17T13:57:32+00:00
NOT-P
DiagnosticReport 2025-12-17T14:07:57+00:00
ICPv4_R221NF2Schwann
R221.1
DiagnosticReport 2025-12-19T17:04:26+00:00
MSI
M220.5
DiagnosticReport 2025-12-17T14:43:08+00:00
ICPv4_R211Polyp
R211.1
DiagnosticReport 2025-12-17T14:57:29+00:00
STORE_TEST
R346.1
DiagnosticReport 2025-12-17T15:05:15+00:00
RT-PCR-QUAN-CBFB::MYH11
M80.12
DiagnosticReport 2025-12-17T15:21:46+00:00
LYMPHOID_QS
M95.6
DiagnosticReport 2025-12-19T12:46:51+00:00
M80_AML_GX
M80.2
DiagnosticReport 2025-12-17T15:53:36+00:00
Fusion_Solid
M217.3
Diagnos

In [18]:
print(len(reports))
df1 = pd.DataFrame([vars(s) for s in reports])
df1

50


,basedOn,category,code,conclusion,conclusionCode,effectiveDateTime,effectivePeriod,encounter,identifier,imagingStudy,...,extension,modifierExtension,text,id,implicitRules,language,meta,_server,_resolved,_owner
0,[<fhirclient.models.fhirreference.FHIRReferenc...,[<fhirclient.models.codeableconcept.CodeableCo...,<fhirclient.models.codeableconcept.CodeableCon...,None,None,None,None,None,[<fhirclient.models.identifier.Identifier obje...,None,...,None,None,None,19,None,None,<fhirclient.models.meta.Meta object at 0x00000...,None,None,None
1,[<fhirclient.models.fhirreference.FHIRReferenc...,[<fhirclient.models.codeableconcept.CodeableCo...,<fhirclient.models.codeableconcept.CodeableCon...,None,None,None,None,None,[<fhirclient.models.identifier.Identifier obje...,None,...,None,None,None,25,None,None,<fhirclient.models.meta.Meta object at 0x00000...,None,None,None
2,[<fhirclient.models.fhirreference.FHIRReferenc...,[<fhirclient.models.codeableconcept.CodeableCo...,<fhirclient.models.codeableconcept.CodeableCon...,None,None,None,None,None,[<fhirclient.models.identifier.Identifier obje...,None,...,[<fhirclient.models.extension.Extension object...,None,None,35,None,None,<fhirclient.models.meta.Meta object at 0x00000...,None,None,None
3,[<fhirclient.models.fhirreference.FHIRReferenc...,[<fhirclient.models.codeableconcept.CodeableCo...,<fhirclient.models.codeableconcept.CodeableCon...,None,None,None,None,None,[<fhirclient.models.identifier.Identifier obje...,None,...,None,None,None,39,None,None,<fhirclient.models.meta.Meta object at 0x00000...,None,None,None
4,[<fhirclient.models.fhirreference.FHIRReferenc...,[<fhirclient.models.codeableconcept.CodeableCo...,<fhirclient.models.codeableconcept.CodeableCon...,None,None,None,None,None,[<fhirclient.models.identifier.Identifier obje...,None,...,None,None,None,44,None,None,<fhirclient.models.meta.Meta object at 0x00000...,None,None,None
5,[<fhirclient.models.fhirreference.FHIRReferenc...,[<fhirclient.models.codeableconcept.CodeableCo...,<fhirclient.models.codeableconcept.CodeableCon...,None,None,None,None,None,[<fhirclient.models.identifier.Identifier obje...,None,...,None,None,None,49,None,None,<fhirclient.models.meta.Meta object at 0x00000...,None,None,None
6,[<fhirclient.models.fhirreference.FHIRReferenc...,[<fhirclient.models.codeableconcept.CodeableCo...,<fhirclient.models.codeableconcept.CodeableCon...,None,None,None,None,None,[<fhirclient.models.identifier.Identifier obje...,None,...,None,None,None,54,None,None,<fhirclient.models.meta.Meta object at 0x00000...,None,None,None
7,[<fhirclient.models.fhirreference.FHIRReferenc...,[<fhirclient.models.codeableconcept.CodeableCo...,<fhirclient.models.codeableconcept.CodeableCon...,None,None,None,None,None,[<fhirclient.models.identifier.Identifier obje...,None,...,[<fhirclient.models.extension.Extension object...,None,None,59,None,None,<fhirclient.models.meta.Meta object at 0x00000...,None,None,None
8,[<fhirclient.models.fhirreference.FHIRReferenc...,[<fhirclient.models.codeableconcept.CodeableCo...,<fhirclient.models.codeableconcept.CodeableCon...,None,None,None,None,None,[<fhirclient.models.identifier.Identifier obje...,None,...,[<fhirclient.models.extension.Extension object...,None,None,64,None,None,<fhirclient.models.meta.Meta object at 0x00000...,None,None,None
9,[<fhirclient.models.fhirreference.FHIRReferenc...,[<fhirclient.models.codeableconcept.CodeableCo...,<fhirclient.models.codeableconcept.CodeableCon...,None,None,None,None,None,[<fhirclient.models.identifier.Identifier obje...,None,...,None,None,None,69,None,None,<fhirclient.models.meta.Meta object at 0x00000...,None,None,None


In [19]:
def performer(my_list):
    performr = ""
    for item in my_list:
        performr = item.display
    return performr

def codeCode(concept):
    code = ""
    for coding in concept.coding:
        code = coding.code

    return code
def codeDisplay(concept):
    code = ""
    for coding in concept.coding:
        code = coding.display

    return code

df1['performerDisplay'] = df1['performer'].apply(performer)
df1['codingCode'] = df1['code'].apply(codeCode)
df1['codingDisplay'] = df1['code'].apply(codeDisplay)

df1

,basedOn,category,code,conclusion,conclusionCode,effectiveDateTime,effectivePeriod,encounter,identifier,imagingStudy,...,id,implicitRules,language,meta,_server,_resolved,_owner,performerDisplay,codingCode,codingDisplay
0,[<fhirclient.models.fhirreference.FHIRReferenc...,[<fhirclient.models.codeableconcept.CodeableCo...,<fhirclient.models.codeableconcept.CodeableCon...,None,None,None,None,None,[<fhirclient.models.identifier.Identifier obje...,None,...,19,None,None,<fhirclient.models.meta.Meta object at 0x00000...,None,None,None,NHS North West Genomics,R31.3_WGS,PANEL: R31.3 - Bilateral congenital or childho...
1,[<fhirclient.models.fhirreference.FHIRReferenc...,[<fhirclient.models.codeableconcept.CodeableCo...,<fhirclient.models.codeableconcept.CodeableCon...,None,None,None,None,None,[<fhirclient.models.identifier.Identifier obje...,None,...,25,None,None,<fhirclient.models.meta.Meta object at 0x00000...,None,None,None,NHS North West Genomics,TARGET_ARRAY_SNP,TEST: Targeted SNP Microarray
2,[<fhirclient.models.fhirreference.FHIRReferenc...,[<fhirclient.models.codeableconcept.CodeableCo...,<fhirclient.models.codeableconcept.CodeableCon...,None,None,None,None,None,[<fhirclient.models.identifier.Identifier obje...,None,...,35,None,None,<fhirclient.models.meta.Meta object at 0x00000...,None,None,None,NHS North West Genomics,M80_AML_GX,PANEL: M80_AML_GX
3,[<fhirclient.models.fhirreference.FHIRReferenc...,[<fhirclient.models.codeableconcept.CodeableCo...,<fhirclient.models.codeableconcept.CodeableCon...,None,None,None,None,None,[<fhirclient.models.identifier.Identifier obje...,None,...,39,None,None,<fhirclient.models.meta.Meta object at 0x00000...,None,None,None,NHS North West Genomics,SANGER Family follow-up Rare,TEST: Sanger Sequencing - Family follow-up tes...
4,[<fhirclient.models.fhirreference.FHIRReferenc...,[<fhirclient.models.codeableconcept.CodeableCo...,<fhirclient.models.codeableconcept.CodeableCon...,None,None,None,None,None,[<fhirclient.models.identifier.Identifier obje...,None,...,44,None,None,<fhirclient.models.meta.Meta object at 0x00000...,None,None,None,NHS North West Genomics,SANGER Diagnostic Rare,TEST: Sanger Sequencing - Diagnostic testing -...
5,[<fhirclient.models.fhirreference.FHIRReferenc...,[<fhirclient.models.codeableconcept.CodeableCo...,<fhirclient.models.codeableconcept.CodeableCon...,None,None,None,None,None,[<fhirclient.models.identifier.Identifier obje...,None,...,49,None,None,<fhirclient.models.meta.Meta object at 0x00000...,None,None,None,NHS North West Genomics,TARGET_ARRAY_SNP,TEST: Targeted SNP Microarray
6,[<fhirclient.models.fhirreference.FHIRReferenc...,[<fhirclient.models.codeableconcept.CodeableCo...,<fhirclient.models.codeableconcept.CodeableCon...,None,None,None,None,None,[<fhirclient.models.identifier.Identifier obje...,None,...,54,None,None,<fhirclient.models.meta.Meta object at 0x00000...,None,None,None,NHS North West Genomics,R125.1,PANEL: R125.1 - Thoracic aortic aneurysm and d...
7,[<fhirclient.models.fhirreference.FHIRReferenc...,[<fhirclient.models.codeableconcept.CodeableCo...,<fhirclient.models.codeableconcept.CodeableCon...,None,None,None,None,None,[<fhirclient.models.identifier.Identifier obje...,None,...,59,None,None,<fhirclient.models.meta.Meta object at 0x00000...,None,None,None,NHS North West Genomics,NOT-P,TEST: Not Processed
8,[<fhirclient.models.fhirreference.FHIRReferenc...,[<fhirclient.models.codeableconcept.CodeableCo...,<fhirclient.models.codeableconcept.CodeableCon...,None,None,None,None,None,[<fhirclient.models.identifier.Identifier obje...,None,...,64,None,None,<fhirclient.models.meta.Meta object at 0x00000...,None,None,None,NHS North West Genomics,R221.1,"PANEL: R221.1, R221.2 - Familial tumours of th..."
9,[<fhirclient.models.fhirreference.FHIRReferenc...,[<fhirclient.models.codeableconcept.CodeableCo...,<fhirclient.models.codeableconcept.CodeableCon...,None,None,None,None,None,[<fhirclient.models.identifier.Identifier obje...,None,...,69,None,None,<fhirclient.m